In [1]:
import cv2
import numpy as np
import tensorflow as tf
from collections import deque

cv2.ocl.setUseOpenCL(False)

def get_age_range(age_class):
    age_ranges = [
        "0-3", "4-7", "8-11", "12-15", "16-19", 
        "20-23", "24-27", "28-31", "32-35", "36-39", 
        "40-43", "44-47", "48-51", "52-55", "56-59", 
        "60-63", "64-67", "68-71", "72-75", "76-79", 
        "80-83", "84-87", "88-91", "92-95", "96-99"
    ]
    return age_ranges[age_class]


emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprise"}

emotion_model = tf.keras.models.load_model('61p_model.h5.keras')
gender_model = tf.keras.models.load_model('gender_classification_model.keras')
age_model = tf.keras.models.load_model('age3_classification_model.keras')

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

emotion_window = deque(maxlen=2)
gender_window = deque(maxlen=10)
age_window = deque(maxlen=10)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]

        if roi_gray_frame.size > 0:
            cropped_img_emotion = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
            emotion_prediction = emotion_model.predict(cropped_img_emotion)
            maxindex_emotion = int(np.argmax(emotion_prediction))

            emotion_window.append(maxindex_emotion)

            if len(emotion_window) == emotion_window.maxlen:
                most_common_emotion = max(set(emotion_window), key=emotion_window.count)
                emotion_text = emotion_dict[most_common_emotion]
            else:
                emotion_text = emotion_dict[maxindex_emotion]

            cropped_img_gender = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
            gender_prediction = gender_model.predict(cropped_img_gender)
            gender_pred = (gender_prediction >= 0.5).astype(int)[:, 0]

            gender_window.append(gender_pred[0])

            if len(gender_window) == gender_window.maxlen:
                most_common_gender = max(set(gender_window), key=gender_window.count)
                gender_text = "Female" if most_common_gender == 0 else "Male"
            else:
                gender_text = "Female" if gender_pred[0] == 0 else "Male"

            cropped_img_age = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
            age_prediction = age_model.predict(cropped_img_age)
            age_class = np.argmax(age_prediction)

            age_window.append(age_class)

            if len(age_window) == age_window.maxlen:
                most_common_age = max(set(age_window), key=age_window.count)
                age_range_text = get_age_range(most_common_age)
            else:
                age_range_text = get_age_range(age_class)

            cv2.putText(frame, f'Emotion: {emotion_text}', (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, f'Gender: {gender_text}', (x+20, y-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, f'Age: {age_range_text}', (x+20, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    height, width = frame.shape[:2]
    new_width = 1200
    new_height = int((height / width) * new_width)
    resized_frame = cv2.resize(frame, (new_width, new_height), interpolation=cv2.INTER_CUBIC)

    cv2.imshow('Video', resized_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 664ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━